In [1]:
import pandas as pd
import matplotlib.pyplot as plt


## Data Exploration

In [2]:
# Read training data from csv
df = pd.read_csv('datasets/train_wiki.csv', usecols=['Title', 'Genre', 'Plot'], na_values='unknown')
df.head()

,Title,Genre,Plot
0,Kansas Saloon Smashers,NaN,"A bartender is working at a saloon, serving dr..."
1,Love by the Light of the Moon,NaN,"The moon, painted with a smiling face hangs ov..."
2,The Martyred Presidents,NaN,"The film, just over a minute long, is composed..."
3,"Terrible Teddy, the Grizzly King",NaN,Lasting just 61 seconds and consisting of two ...
4,Jack and the Beanstalk,NaN,The earliest known adaptation of the classic f...


In [57]:
# Strip whitespace, remove extraneous genres if more than one per row
def clean_genre(s):
    delims = ('/', ',', '(', '[')
    for delim in delims:
        idx = s.find(delim)
        if not idx == -1:
            s = s[:idx]
    return s.strip()

# Remove rows with no genre
df = df[pd.notnull(df['Genre'])]
# Clean Genre column
df['Genre'] = df['Genre'].map(clean_genre)
# Remove rows where genre is less than 2 letters
df = df[df['Genre'].map(lambda x: len(x) > 1)]
# Generate column of genre_ids
df['genre_id'] = df['Genre'].factorize()[0]
# DF to keep track of genre and respective id
genre_id_df = df[['Genre', 'genre_id']].drop_duplicates().sort_values('genre_id')
# dicts for quick genre id lookup
genre_to_id = dict(genre_id_df.values)
id_to_category = dict(genre_id_df[['genre_id', 'Genre']].values)

df.head()


,Title,Genre,Plot,genre_id
6,The Suburbanite,comedy,The film is about a family who move to the sub...,0
9,Dream of a Rarebit Fiend,short,The Rarebit Fiend gorges on Welsh rarebit at a...,1
10,From Leadville to Aspen: A Hold-Up in the Rockies,short action,The film features a train traveling through th...,2
11,Kathleen Mavourneen,short film,Irish villager Kathleen is a tenant of Captain...,3
12,Daniel Boone,biographical,Boone's daughter befriends an Indian maiden as...,4


## Imbalanced Classes

In [47]:
# Q: How is the distribution of movie genres?
genre_counts = df.groupby('Genre').Plot.count()
plt.hist(genre_counts)
plt.title('Movie genre distribution')
plt.ylabel('Frequency')
plt.xlabel('Number of movies per genre')
plt.show()

min_movies = 2
unpop_genres = (df.groupby('Genre').Plot.count() < min_movies).sum()
num_genres = len(genre_id_df)
print("{:.2%} ({}) of {} genres have less than {} movies".format(unpop_genres/num_genres, unpop_genres
                                                              , num_genres, min_movies))
# A: Most genres have very few movies
print("The most popular genre, {}, has {} movies".format())


60.30% (480) of 796 genres have less than 2 movies


Most genres have <2 movies, and the most popular genre, 

In [64]:
df.groupby('Genre').Plot.count().sort_values(ascending=False)

Genre
drama                                                       6160
comedy                                                      4745
action                                                      1617
horror                                                      1191
romance                                                     1101
thriller                                                     940
western                                                      804
crime                                                        629
adventure                                                    541
musical                                                      470
romantic comedy                                              426
crime drama                                                  422
science fiction                                              409
mystery                                                      314
film noir                                                    298
animation          